<a href="https://colab.research.google.com/github/95871abhinandankumar/95871abhinandankumar/blob/main/assignmant7e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from sklearn import datasets
import numpy as np
import math
import pandas as pd
iris = datasets.load_iris()

In [54]:
print("target name")
print(iris.target_names)
print("target...")
iris.target

target name
['setosa' 'versicolor' 'virginica']
target...


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [55]:
df = pd.DataFrame(iris.data,columns=[iris.feature_names])
df['target'] = iris.target

print(df)

    sepal length (cm) sepal width (cm)  ... petal width (cm) target
0                 5.1              3.5  ...              0.2      0
1                 4.9              3.0  ...              0.2      0
2                 4.7              3.2  ...              0.2      0
3                 4.6              3.1  ...              0.2      0
4                 5.0              3.6  ...              0.2      0
..                ...              ...  ...              ...    ...
145               6.7              3.0  ...              2.3      2
146               6.3              2.5  ...              1.9      2
147               6.5              3.0  ...              2.0      2
148               6.2              3.4  ...              2.3      2
149               5.9              3.0  ...              1.8      2

[150 rows x 5 columns]


In [56]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler()
samples = scaling.fit_transform(df[df.columns])
df = pd.DataFrame(samples)

print(df)

            0         1         2         3    4
0    0.222222  0.625000  0.067797  0.041667  0.0
1    0.166667  0.416667  0.067797  0.041667  0.0
2    0.111111  0.500000  0.050847  0.041667  0.0
3    0.083333  0.458333  0.084746  0.041667  0.0
4    0.194444  0.666667  0.067797  0.041667  0.0
..        ...       ...       ...       ...  ...
145  0.666667  0.416667  0.711864  0.916667  1.0
146  0.555556  0.208333  0.677966  0.750000  1.0
147  0.611111  0.416667  0.711864  0.791667  1.0
148  0.527778  0.583333  0.745763  0.916667  1.0
149  0.444444  0.416667  0.694915  0.708333  1.0

[150 rows x 5 columns]


In [57]:
def hypothesis(w,dataset):
  y=w[0]
  sample =dataset[:-1]
  sample = np.concatenate([[1], sample])
  y = np.multiply(w, sample)
  y = np.sum(y)
  gx = 1/(1+(math.e)**(-1*y))
  return gx

In [58]:
def predicted_values(w, dataset):
  predicted = []
  for i in range(len(dataset)):
    predicted.append(hypothesis(w, dataset[i]))
  
  return np.array(predicted)


# to calculate mean square error
def MSE(predicted, actual):
  return np.square(np.subtract(predicted, actual)).mean()/2

print(MSE(np.array([1,2,3,4]), np.array([1,2,3,4]),))


def accurracy(predicted, actual):
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  for i in range(len(predicted)):
    if predicted[i] >= 0.5 and actual[i] == 1:
      tp += 1
    elif predicted[i] >= 0.5 and actual[i] == 0:
      fp += 1
    elif predicted[i] <= 0.5 and actual[i] == 0:
      tn += 1
    elif predicted[i] <= 0.5 and actual[i] == 0:
      fn += 1
  return tp, tn, fp, fn

0.0


In [59]:
# to update w using stochastic gradient decendent
def update_parameter_stochastic(w, alpha, dataset):
  np.random.shuffle(dataset)
  m = len(dataset)
  for j in range(m):
    hy = hypothesis(w, np.array(dataset[j]))
    w[0] = w[0] - (alpha /m)*(hy- dataset[:, -1][j])
    for i in range(1, len(w)):
      w[i] = w[i] - (alpha /m)*((hy- dataset[:, -1][j])* dataset[:, i-1][j])
  return w

In [60]:
def logistic_regression(epoch, alpha, rho, w, train_dataset):
  pre_MSE = 0
  curr_MSE = 0
  trains_MSE=[]
  for itr in range(epoch):
    predicted = predicted_values(w, train_dataset)
    curr_MSE = MSE(predicted, train_dataset[:,-1])
    w = update_parameter_stochastic(w, alpha, train_dataset)
    
    trains_MSE.append(curr_MSE)

    if abs(curr_MSE - pre_MSE) <= rho:
      break;
    pre_MSE = curr_MSE
  return w, trains_MSE

In [61]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [68]:
dataset = df.to_numpy()
kf = KFold(n_splits=5,shuffle=True)
w_versicolor = [1,1,1,1,1]
w_verginica = [1,1,1,1,1]
w_setosa = [1,1,1,1,1]

for train_index, test_index in kf.split(dataset):
  print("  ..............................  ")
  train = []
  test = []
  for i in train_index :
    train.append(dataset[i])

  for i in test_index :
    test.append(dataset[i])


  train_setosa = np.copy(train)

  for i in range(len(train_setosa)):
    if train_setosa[i][-1] == 0:
      train_setosa[i][-1] = 1
    else:
      train_setosa[i][-1] = 0
  
  test_setosa = np.copy(test)

  for i in range(len(test_setosa)):
    if test_setosa[i][-1] == 0:
      test_setosa[i][-1] = 1
    else:
      test_setosa[i][-1] = 0

  train_versicolor = np.copy(train)

  for i in range(len(train_versicolor)):
    if train_versicolor[i][-1] == 0.5:
      train_versicolor[i][-1] = 1
    else:
      train_versicolor[i][-1] = 0
  
  test_versicolor = np.copy(test)

  for i in range(len(test_versicolor)):
    if test_versicolor[i][-1] == 0.5:
      test_versicolor[i][-1] = 1
    else:
      test_versicolor[i][-1] = 0

  train_verginica = np.copy(train)

  for i in range(len(train_verginica)):
    if train_verginica[i][-1] != 1:
      train_verginica[i][-1] = 0


  test_verginica = np.copy(test)

  for i in range(len(test_verginica)):
    if test_verginica[i][-1] != 1:
      test_verginica[i][-1] = 0

  print("class wise accurracy  for setosa ")
  w_setosa, st_mse= logistic_regression(7, i, 0.1, w_setosa, train_setosa)
  predicted = predicted_values(w_setosa, train_setosa)
  tp, tn, fp, fn = accurracy(predicted, train_setosa[:,-1])
  print("train accurracy of setosa", (tp + tn )/len(train_setosa))
  print("precison " , tp/len(train_setosa))
  print("recall ", tp/len(train_setosa))
  predicted = predicted_values(w_setosa, test_setosa)
  tp, tn, fp, fn = accurracy(predicted, test_setosa[:,-1])
  tp_st = tp
  print("test accurracy of setosa", (tp + tn )/len(test_setosa))
  print("precison " , tp/len(train_setosa))
  print("recall ", tp/len(train_setosa))

  w_versicolor, st_mse= logistic_regression(7, i, 0.1, w_versicolor, train_versicolor)
  predicted = predicted_values(w_versicolor, train_versicolor)
  tp, tn, fp, fn = accurracy(predicted, train_versicolor[:,-1])
  print("train accurracy of versicolor", (tp + tn )/len(train_versicolor))
  print("precison " , tp/len(train_versicolor))
  print("recall ", tp/len(train_versicolor))
  predicted = predicted_values(w_versicolor, test_versicolor)
  tp, tn, fp, fn = accurracy(predicted, test_versicolor[:,-1])
  print("test accurracy of versicolor", (tp + tn )/len(test_versicolor))
  print("precison " , tp/len(train_versicolor))
  print("recall ", tp/len(train_versicolor))
  tp_tv = tp

  w_verginica, st_mse= logistic_regression(7, i, 0.1, w_verginica, train_verginica)
  predicted = predicted_values(w_verginica, train_verginica)
  tp, tn, fp, fn = accurracy(predicted, train_verginica[:,-1])
  print("train accurracy of verginica", (tp + tn )/len(train_verginica))
  print("precison " , tp/len(train_verginica))
  print("recall ", tp/len(train_verginica))
  predicted = predicted_values(w_verginica, test_verginica)
  tp, tn, fp, fn = accurracy(predicted, test_verginica[:,-1])
  print("test accurracy of verginica", (tp + tn )/len(test_verginica))
  print("precison " , tp/len(train_verginica))
  print("recall ", tp/(len(train_verginica)))
  tp_vt = tp

  print("overall accuraccy : ", (tp_st + tp_tv + tp_vt)/(len(test)))
    
  

  ..............................  
class wise accurracy  for setosa 
train accurracy of setosa 1.0
precison  0.325
recall  0.325
test accurracy of setosa 1.0
precison  0.09166666666666666
recall  0.09166666666666666
train accurracy of versicolor 0.6333333333333333
precison  0.1
recall  0.1
test accurracy of versicolor 0.6666666666666666
precison  0.016666666666666666
recall  0.016666666666666666
train accurracy of verginica 0.9666666666666667
precison  0.30833333333333335
recall  0.30833333333333335
test accurracy of verginica 1.0
precison  0.09166666666666666
recall  0.09166666666666666
overall accuraccy :  0.8
  ..............................  
class wise accurracy  for setosa 
train accurracy of setosa 1.0
precison  0.375
recall  0.375
test accurracy of setosa 1.0
precison  0.041666666666666664
recall  0.041666666666666664
train accurracy of versicolor 0.675
precison  0.03333333333333333
recall  0.03333333333333333
test accurracy of versicolor 0.6
precison  0.016666666666666666
reca